<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/rsixu100kompozit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Gerekli kütüphaneleri yükleyin
!pip install git+https://github.com/rongardF/tvdatafeed tradingview-screener openpyxl statsmodels tqdm

import pandas as pd
import numpy as np
from tvDatafeed import TvDatafeed, Interval
from tqdm import tqdm
from tradingview_screener import get_all_symbols
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

# RSI'yi RMA (SMMA) ile hesaplama fonksiyonu
def calculate_rsi_rma(data, length=14):
    delta = data['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.ewm(com=(length - 1), min_periods=length).mean()
    avg_loss = loss.ewm(com=(length - 1), min_periods=length).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Haftalık tarama yapmak için sabit zaman dilimi
def get_weekly_interval():
    return 'Haftalık', Interval.in_weekly

# BIST'teki tüm hisse senetlerini otomatik çekme
def get_all_stocks():
    # tradingview_screener'dan tüm hisseleri al
    tv = TvDatafeed()  # TVDatafeed örneği oluştur
    Hisseler = get_all_symbols(market='turkey')  # Türk hisselerini al
    Hisseler = [symbol.replace('BIST:', '') for symbol in Hisseler]  # BIST: kısmını kaldır
    return sorted(Hisseler)  # Sıralı liste döndür

# Veriyi çekme ve RSI 50 kırılımı analizi yapma
def analyze_rsi_breakout(stock_list):
    interval_name, interval = get_weekly_interval()
    tv = TvDatafeed()

    # XU100 verisini haftalık olarak çek
    xu100_data = tv.get_hist('XU100', exchange='BIST', interval=interval, n_bars=1000)
    xu100_data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)

    results = []
    for stock in tqdm(stock_list, desc="Taramalar ilerliyor"):
        try:
            data = tv.get_hist(symbol=f"{stock}/XU100", exchange='BIST', interval=interval, n_bars=1000)
            data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
            data['RSI'] = calculate_rsi_rma(data)

            if data['RSI'].iloc[-2] < 50 <= data['RSI'].iloc[-1]:
                results.append([stock, data['Close'].iloc[-1], "RSI 50 Üzeri", interval_name, data.index[-1]])
                print(f"{stock} - RSI 50 kırılımı - Zaman Dilimi: {interval_name}")

        except Exception as e:
            print(f"{stock} işlenirken hata: {e}")

    df_results = pd.DataFrame(results, columns=['Hisse Adı', 'Son Fiyat', 'Kırılım Türü', 'Zaman Dilimi', 'Kırılım Zamanı'])
    return df_results

# Tüm hisseleri çek
stock_list = get_all_stocks()

# Tarama yap ve sonuçları göster
df_results = analyze_rsi_breakout(stock_list)
print("RSI 50 Kırılım Sinyalleri:")
print(df_results)

# Excel dosyasına kaydet
excel_file = 'rsi_breakout_signals_weekly.xlsx'
df_results.to_excel(excel_file, index=False)
print(f"RSI 50 kırılım sinyalleri '{excel_file}' dosyasına kaydedildi.")

  Cloning https://github.com/rongardF/tvdatafeed to /tmp/pip-req-build-fvkd82m5
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed /tmp/pip-req-build-fvkd82m5
  Resolved https://github.com/rongardF/tvdatafeed to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.4 MB/s eta 0:00:00
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=22b0268b19cee7dc00db81a250dc02d9dc75ad3c826c76c32b62d348a45d8ed5
  Stored in directory: /tmp/pip-ephem-wheel-cache-zaaoaqhg/wheels/e4/32/1e/21ebcacc6549d75fae3bf3ff75cee8fcbe4e5c189d88fbd5a1
Successfully built tvdatafeed


Taramalar ilerliyor:   6%|▌         | 32/580 [00:28<09:00,  1.01it/s]

ALKA - RSI 50 kırılımı - Zaman Dilimi: Haftalık


Taramalar ilerliyor:  17%|█▋        | 101/580 [01:07<06:04,  1.31it/s]

BRISA - RSI 50 kırılımı - Zaman Dilimi: Haftalık


...

SUWEN - RSI 50 kırılımı - Zaman Dilimi: Haftalık


Taramalar ilerliyor:  98%|█████████▊| 571/580 [05:47<00:08,  1.04it/s]

YYAPI - RSI 50 kırılımı - Zaman Dilimi: Haftalık


Taramalar ilerliyor: 100%|██████████| 580/580 [05:53<00:00,  1.64it/s]


RSI 50 Kırılım Sinyalleri:
   Hisse Adı  Son Fiyat  Kırılım Türü Zaman Dilimi      Kırılım Zamanı
0       ALKA   0.002881  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
1      BRISA   0.010092  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
2      BVSAN   0.009862  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
3      GLBMD   0.001267  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
4      IMASM   0.000329  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
5      ISCTR   0.001365  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
6      KIMMR   0.001038  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
7      KRDMB   0.002178  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
8      KRDMD   0.002829  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
9      KTSKR   0.006819  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
10     KUVVA   0.004777  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
11     SUWEN   0.002967  RSI 50 Üzeri     Haftalık 2024-11-18 06:00:00
12     YYAPI   0.000528  RSI 50 Üzeri     Haftalık